In [1]:
!ls -l TeraGen-1TB

total 2929694400
-rw-r--r-- 1 seff-34c2f0d3dcc620-a916a00b641d users 3333333300 Jun 29 13:27 part-r-00000
-rw-r--r-- 1 seff-34c2f0d3dcc620-a916a00b641d users 3333333300 Jun 29 13:25 part-r-00001
-rw-r--r-- 1 seff-34c2f0d3dcc620-a916a00b641d users 3333333300 Jun 29 13:35 part-r-00002
-rw-r--r-- 1 seff-34c2f0d3dcc620-a916a00b641d users 3333333300 Jun 29 13:31 part-r-00003
-rw-r--r-- 1 seff-34c2f0d3dcc620-a916a00b641d users 3333333300 Jun 29 13:33 part-r-00004
-rw-r--r-- 1 seff-34c2f0d3dcc620-a916a00b641d users 3333333300 Jun 29 13:26 part-r-00005
-rw-r--r-- 1 seff-34c2f0d3dcc620-a916a00b641d users 3333333300 Jun 29 13:35 part-r-00006
-rw-r--r-- 1 seff-34c2f0d3dcc620-a916a00b641d users 3333333300 Jun 29 13:37 part-r-00007
-rw-r--r-- 1 seff-34c2f0d3dcc620-a916a00b641d users 3333333300 Jun 29 13:37 part-r-00008
-rw-r--r-- 1 seff-34c2f0d3dcc620-a916a00b641d users 3333333300 Jun 29 13:37 part-r-00009
-rw-r--r-- 1 seff-34c2f0d3dcc620-a916a00b641d users 3333333300 Jun 29 13:22 part-r-00010
-rw-

In [2]:
teraFile = spark.read.text("TeraGen-1TB/")

In [3]:
#teraFile.sample(True,.3).show()
teraFile.groupBy("value").count().take(100)

KeyboardInterrupt: 

In [1]:
from pyspark.sql.types import LongType
from pyspark.sql.types import StringType
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import lit
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import regexp_replace
from pyspark.ml.feature import PCA
from pyspark.ml.feature import Normalizer
from pyspark.ml.linalg import Vectors
from pyspark.ml.clustering import KMeans
from pyspark.sql.functions import rand, randn

In [3]:
def setHadoopConfig(name):
    prefix = "fs.swift2d.service." + name
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + '.auth.url', 'https://identity.open.softlayer.com'+'/v3/auth/tokens')
    hconf.set(prefix + '.auth.endpoint.prefix', 'endpoints')
    hconf.set(prefix + '.tenant', 'a9fb4d478e3d40a8bbd54c5a2ecf25a3')
    hconf.set(prefix + '.username', '6a4cc8251c1940179a6cccc9098a15e0')
    hconf.set(prefix + '.password', 'kDTcKA2H(3eo5.G0')
    hconf.setInt(prefix + '.http.port', 8080)
    hconf.set(prefix + '.region', 'dallas')
    hconf.setBoolean(prefix + '.public', False)

name = "keystone"
setHadoopConfig(name)

seven_cases = spark.read.parquet("swift2d://MGH." + name + "/tempParq/7cases.parquet")

cases = spark.read.csv("swift2d://MGH." + name + "/csv/csv_case1.csv",
                           header=False,inferSchema=True,nanValue=0,nullValue=0,negativeInf=0,positiveInf=0)\
                        .withColumn("timeId", monotonically_increasing_id().cast(LongType()))\
                        .withColumn("patId", lit(1).cast(LongType()))

In [4]:
caseNumbers = [2,8,9,10,11,12]
for num in caseNumbers:
    caseTemp = spark.read.csv("swift2d://MGH." + name + "/csv/csv_case"+str(num)+".csv",
                           header=False,inferSchema=True,nanValue=0,nullValue=0,negativeInf=0,positiveInf=0)\
                        .withColumn("timeId", monotonically_increasing_id().cast(LongType()))\
                        .withColumn("patId", lit(1).cast(LongType()))
    cases = cases.union(caseTemp)

In [5]:
for col in cases.schema:
    #print col.dataType
    #print StringType
    if str(col.dataType) == "StringType":
        #print col
        cases = cases.withColumn(col.name + "temp", 
                                 regexp_replace(
                                     regexp_replace(cases[col.name],"Inf","0")
                                     ,"NaN","0"))
        cases = cases.withColumn(col.name + "temp", cases[col.name + "temp"].cast(DoubleType()))
        cases = cases.drop(col.name).withColumnRenamed(col.name + "temp",col.name)

cases = cases.na.fill(0)

In [6]:
feature_cols = [i for i in cases.columns if i != "timeId" and i != "patId"]
assembler = VectorAssembler().setInputCols(feature_cols).setOutputCol("features")
cases = assembler.transform(cases)

In [7]:
normalizer = Normalizer().setInputCol("features").setOutputCol("normFeatures")
cases = normalizer.transform(cases)

pcaModel = PCA().setInputCol("normFeatures").setOutputCol("pcaFeatures").setK(100).fit(cases)

cases = pcaModel.transform(cases)
cases = cases.cache()

In [8]:
kmeans = KMeans().setFeaturesCol("pcaFeatures").setK(18)
kmeansModel = kmeans.fit(cases)
cases = kmeansModel.transform(cases)

In [9]:
cases.repartition(150)
seven_cases.repartition(150)
print

In [10]:
print(cases.take(2))
print(seven_cases.take(2))

[Row(_c0=2.2543, _c1=2.2802, _c2=3.2058, _c3=2.7831, _c4=2.2414, _c5=2.3961, _c6=3.1904, _c7=2.8007, _c8=2.2199, _c9=2.3412, _c10=3.0924, _c11=2.7539, _c12=1.973, _c13=2.2811, _c14=3.0617, _c15=2.9103, _c30=3.0422, _c31=3.6411, _c32=254330.0, _c33=226880.0, _c34=341440.0, _c35=284210.0, _c36=109680.0, _c37=113230.0, _c38=125590.0, _c39=124770.0, _c40=53409.0, _c41=56575.0, _c42=60883.0, _c43=60030.0, _c44=12915.0, _c45=13836.0, _c46=13558.0, _c47=15694.0, _c48=62.59, _c49=56.045, _c50=118.16, _c51=97.662, _c52=49.497, _c53=53.792, _c54=87.745, _c55=83.527, _c56=47.886, _c57=54.141, _c58=82.052, _c59=77.989, _c60=42.201, _c61=45.07, _c62=73.033, _c63=81.49, _c64=230.36, _c65=149.73, _c66=464.07, _c67=249.64, _c68=74.399, _c69=79.464, _c70=128.04, _c71=125.21, _c72=72.525, _c73=80.443, _c74=116.56, _c75=114.27, _c76=60.959, _c77=59.875, _c78=98.354, _c79=111.16, _c80=0.8756, _c81=0.8617, _c82=0.8544, _c83=0.88318, _c84=0.85872, _c85=0.84645, _c86=0.80648, _c87=0.85755, _c88=0.85065, _c89

In [11]:
df = spark.range(1000000000)

In [12]:
df = df.select("id", rand().alias("uniform"), randn().alias("normal")).where("normal < 2")

In [13]:
df.cache()
df.count()

977257197

In [14]:
cases.count()

247625

In [15]:
df.show()

+---+--------------------+--------------------+
| id|             uniform|              normal|
+---+--------------------+--------------------+
|  0|  0.3323563249143172|-0.32417782915056564|
|  1|  0.8472827156135285|-0.24745971007045106|
|  2| 0.15348734756463034| -0.2524582425739663|
|  3|  0.1508886366881138| -1.4453912729502667|
|  4| 0.22173651282443962| -1.3420700641179573|
|  6| 0.26058602331357983|  -0.156608792344244|
|  7|0.015107226206461544|  0.5525389729540963|
|  8|  0.8387102875652054|-0.38644305100492354|
|  9|  0.9760908140617254|-0.01319135084762...|
| 10|   0.742833999505326| 0.11057550306586095|
| 11|  0.6872343219568114|-0.11583711346555077|
| 12| 0.24999852837560665|  0.4125825645474328|
| 13|  0.8997739929120163| 0.34575136370636356|
| 14|  0.9792427467613107|  1.4334583870276274|
| 15|   0.617467437481665| -0.7307191798467972|
| 17| 0.09417484125085063| 0.10014550440595116|
| 18|  0.4255071281008658| -1.4690917623626116|
| 19|  0.5222144081142351| 0.21752907795

In [16]:
spark.conf.get("spark.executor.memory")

u'6G'

In [17]:
sc._conf.getAll()

[(u'spark.eventLog.enabled', u'true'),
 (u'spark.deploy.resourceScheduler.factory',
  u'org.apache.spark.deploy.master.EGOResourceSchedulerFactory'),
 (u'spark.app.id',
  u'app-20170629103350-1109-f751dae1-8e9d-42cb-991f-0f7deed1cf21'),
 (u'spark.r.command',
  u'/usr/local/src/bluemix_jupyter_bundle.v48/R/bin/Rscript'),
 (u'spark.master', u'spark://yp-spark-dal09-env5-0021:7089'),
 (u'spark.executor.memory', u'6G'),
 (u'spark.driver.maxResultSize', u'13G'),
 (u'spark.shuffle.service.port', u'7342'),
 (u'spark.extraListeners', u'com.ibm.spaas.listeners.DB2DialectRegistrar'),
 (u'spark.serializer.objectStreamReset', u'100'),
 (u'spark.submit.deployMode', u'client'),
 (u'spark.driver.host', u'10.142.18.236'),
 (u'spark.driver.memory', u'16384M'),
 (u'spark.worker.ui.retainedExecutors', u'0'),
 (u'spark.ui.retainedJobs', u'0'),
 (u'spark.shuffle.service.enabled', u'true'),
 (u'spark.executor.extraJavaOptions',
  u'-Djava.security.egd=file:/dev/./urandom'),
 (u'spark.executor.id', u'driver'

In [ ]:
print 1

In [1]:
!ls -l

total 11967144
-rw------- 1 seff-34c2f0d3dcc620-a916a00b641d users 1478690037 Jun 27 15:58 Case18_seg1.mat
-rw------- 1 seff-34c2f0d3dcc620-a916a00b641d users  733859235 Jun 28 12:38 Case1_seg11.mat
-rw------- 1 seff-34c2f0d3dcc620-a916a00b641d users   57286534 Jun 29 15:28 Case1_seg12.mat
-rw------- 1 seff-34c2f0d3dcc620-a916a00b641d users  983503825 Jun 27 16:44 Case1_seg1.mat
-rw------- 1 seff-34c2f0d3dcc620-a916a00b641d users  826333198 Jun 27 16:19 Case5_seg1.mat
-rw------- 1 seff-34c2f0d3dcc620-a916a00b641d users       4329 Jun 28 13:18 compute_spectrogram_sunhaoqi.py
-rw------- 1 seff-34c2f0d3dcc620-a916a00b641d users       3569 Jun 28 13:58 compute_spectrogram_sunhaoqi.pyc
drwx------ 2 seff-34c2f0d3dcc620-a916a00b641d users       4096 Jun 29 12:42 debug
-rw------- 1 seff-34c2f0d3dcc620-a916a00b641d users        703 Jun 29 08:46 fcn_shannon_entro.py
-rw------- 1 seff-34c2f0d3dcc620-a916a00b641d users      15695 Jun 29 15:27 full_pipeline.py
-rw------- 1 seff-34c2f0d3dcc620-a916a